In [1]:
!pip install transformers
!pip install datasets
!pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 83.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 21.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 15.5 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-whe

In [2]:
from datasets import load_metric
metric = load_metric("rouge")

<ipython-input-2-ffe5587f2e49>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")


# **Data Test Preparation**

In [3]:
from datasets import load_dataset,Dataset

dataset = load_dataset("csebuetnlp/xlsum","english")
dataset_test = dataset.get("test")

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset xlsum downloaded and prepared to /root/.cache/huggingface/datasets/csebuetnlp___xlsum/english/2.0.0/518ab0af76048660bcc2240ca6e8692a977c80e384ffb18fdddebaca6daebdce. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

# **Evaluasi Data**
- https://medium.com/nlplanet/a-full-guide-to-finetuning-t5-for-text2text-and-building-a-demo-with-streamlit-c72009631887

In [4]:
from transformers import AutoTokenizer,AutoModelForSeq2SeqLM, EvalPrediction

In [5]:
model = AutoModelForSeq2SeqLM.from_pretrained("fanzru/t5-small-finetuned-xlsum-with-multi-news-10-epoch")
tokenizer = AutoTokenizer.from_pretrained("fanzru/t5-small-finetuned-xlsum-with-multi-news-10-epoch")

Downloading:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/242M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

In [6]:
import nltk
nltk.download('punkt')
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [7]:
goverment_politic_key = ["goverment", "politic", "election","political","parties", "political party"]
health_key = ["covid-19", "corona", "health", "covid", "vaccine", "medicine", "aids", "hiv", "virus"]
economic_key = ["economy", "economic", "recession", "inflation", "economical"]
infrastucture_development_key = ["infrastructure", "facilities", "train", "development","construction"]
crime_key = ["crime", "robbery", "theft", "sexual harassement","illegal", "drug","criminal" "law","criminal"]
tech_industry_key = ["tech", "technology", "industry", "startup", "start-up", "business"]

In [8]:
import pandas as pd
import re

def insert_new_data(key,df_source,df_new,col):
  df_pd_new = df_new
  idx=1
  for value in df_source:
    key_found = False
    for k in key:
      if re.findall(k, value[col]):
        key_found = True
        break 
    if key_found:
      new_value = pd.DataFrame({"id":value.get("id") ,"url":value.get("url"), "title":value.get("title"), "text": value.get("text"), "summary": value.get("summary")},index=[idx])
      idx = idx +1
      df_pd_new = df_pd_new.append(new_value)

  return Dataset.from_pandas(df_pd_new)

In [ ]:
def get_non_clasification_news(df):
  sum_news_non_clasification = 0
  idx=1
  df_pd_new = pd.DataFrame(columns=['id', 'url', 'title', 'summary', 'text']) 
  for value in df:
    
    key_found = False
    
    for k in goverment_politic_key:
      if re.findall(k, value["text"]):
        key_found = True
        break
    if key_found:
        continue
    for k in health_key:
      if re.findall(k, value["text"]):
        key_found = True
        break
    if key_found:
        continue
    for k in economic_key:
      if re.findall(k, value["text"]):
        key_found = True
        break
    if key_found:
        continue

    for k in infrastucture_development_key:
      if re.findall(k, value["text"]):
        key_found = True
        break
    if key_found:
        continue

    for k in tech_industry_key:
      if re.findall(k, value["text"]):
        key_found = True
        break
    if key_found:
        continue

    new_value = pd.DataFrame({"id":value.get("id") ,"url":value.get("url"), "title":value.get("title"), "text": value.get("text"), "summary": value.get("summary")},index=[idx])
    idx = idx +1
    df_pd_new = df_pd_new.append(new_value)
    

    sum_news_non_clasification+=1
  print(f'Sum news non clasification {sum_news_non_clasification} from {len(df)} XLSum')
  return Dataset.from_pandas(df_pd_new)

In [9]:
import copy
from datasets import Dataset
df_train_xlsum = copy.deepcopy(dataset["test"])

In [ ]:
df_train_xlsum

Dataset({
    features: ['id', 'url', 'title', 'summary', 'text'],
    num_rows: 11535
})

In [ ]:
# Crime
df = pd.DataFrame(columns=['id', 'url', 'title', 'summary', 'text']) 
df_test_crime = insert_new_data(crime_key, df_train_xlsum,df,"text")
print(f'lenght dataset crime                : {len(df_test_crime)}')
# Health
df = pd.DataFrame(columns=['id', 'url', 'title', 'summary', 'text']) 
df_test_health = insert_new_data(health_key, df_train_xlsum,df,"text")
print(f'lenght dataset health               : {len(df_test_health)}')
# Economic
df = pd.DataFrame(columns=['id', 'url', 'title', 'summary', 'text']) 
df_test_economic = insert_new_data(economic_key, df_train_xlsum,df,"text")
print(f'lenght dataset economic             : {len(df_test_economic)}')
# Infrastucture Development
df = pd.DataFrame(columns=['id', 'url', 'title', 'summary', 'text']) 
df_test_infrastucture_development = insert_new_data(infrastucture_development_key, df_train_xlsum,df,"text")
print(f'lenght dataset infrastructure dev   : {len(df_test_infrastucture_development)}')
# Tech Industry
df = pd.DataFrame(columns=['id', 'url', 'title', 'summary', 'text']) 
df_test_tech_industry = insert_new_data(tech_industry_key, df_train_xlsum,df,"text")
print(f'lenght dataset tech industry        : {len(df_test_tech_industry)}')

lenght dataset crime                : 1776
lenght dataset health               : 2398
lenght dataset economic             : 1405
lenght dataset infrastructure dev   : 1622
lenght dataset tech industry        : 2896


In [12]:
import torch
prefix = "summarize: "
max_input_length = 1024
max_target_length = 128

def evaluation_data(df_test_eval):
  # get test split
  test_tokenized_dataset = df_test_eval
  # pad texts to the same length
  def preprocess_test(examples):
    inputs = [prefix + text for text in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True,
                            padding="max_length")
    return model_inputs

  test_tokenized_dataset = test_tokenized_dataset.map(preprocess_test, batched=True)

  # prepare dataloader
  test_tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])
  dataloader = torch.utils.data.DataLoader(test_tokenized_dataset, batch_size=16)
  print(dataloader)
  # generate text for each batch
  all_predictions = []
  for i,batch in enumerate(dataloader):
    predictions = model.generate(**batch)
    all_predictions.append(predictions)

  # flatten predictions
  all_predictions_flattened = [pred for preds in all_predictions for pred in preds]

  # tokenize and pad titles
  all_summary = tokenizer(test_tokenized_dataset["summary"], max_length=max_target_length,
                        truncation=True, padding="max_length")["input_ids"]

  # compute metrics
  predictions_labels = [all_predictions_flattened, all_summary]
  return compute_metrics(predictions_labels)


In [ ]:
print("---------------------------------")
result_crime = evaluation_data(df_test_crime)
print("Rouge Data Crime")
print(f'Result           : {result_crime}')
print("---------------------------------")

---------------------------------


  0%|          | 0/2 [00:00<?, ?ba/s]

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1387: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Rouge Data Crime
Result           : {'rouge1': 31.0676, 'rouge2': 9.9291, 'rougeL': 24.2629, 'rougeLsum': 24.3435, 'gen_len': 18.8587}
---------------------------------


In [ ]:
print("---------------------------------")
result_health = evaluation_data(df_test_health)
print("Rouge Data Health")
print(f'Result           : {result_health}')
print("---------------------------------")

---------------------------------


  0%|          | 0/3 [00:00<?, ?ba/s]

Rouge Data Health
Result           : {'rouge1': 29.7895, 'rouge2': 9.0008, 'rougeL': 23.2296, 'rougeLsum': 23.3414, 'gen_len': 18.844}
---------------------------------


In [ ]:
print("---------------------------------")
result_economic = evaluation_data(df_test_economic)
print("Rouge Data Economic")
print(f'Result           : {result_economic}')
print("---------------------------------")

---------------------------------


  0%|          | 0/2 [00:00<?, ?ba/s]

Rouge Data Economic
Result           : {'rouge1': 29.6269, 'rouge2': 9.3641, 'rougeL': 23.2046, 'rougeLsum': 23.3023, 'gen_len': 18.7943}
---------------------------------


In [ ]:
print("---------------------------------")
result_infra_dev = evaluation_data(df_test_infrastucture_development)
print("Rouge Data Infrastructure Dev")
print(f'Result           : {result_infra_dev}')
print("---------------------------------")

---------------------------------


  0%|          | 0/2 [00:00<?, ?ba/s]

Rouge Data Infrastructure Dev
Result           : {'rouge1': 29.8256, 'rouge2': 8.6619, 'rougeL': 23.343, 'rougeLsum': 23.4239, 'gen_len': 18.8113}
---------------------------------


In [ ]:
print("---------------------------------")
result_tech_industry = evaluation_data(df_test_tech_industry)
print("Rouge Data Tech Industry")
print(f'Result           : {result_tech_industry}')
print("---------------------------------")

---------------------------------


  0%|          | 0/3 [00:00<?, ?ba/s]

Rouge Data Tech Industry
Result           : {'rouge1': 29.8679, 'rouge2': 8.9511, 'rougeL': 23.3813, 'rougeLsum': 23.477, 'gen_len': 18.7977}
---------------------------------


In [ ]:
print("---------------------------------")
df_non_class = get_non_clasification_news(dataset_test)
result_non_class = evaluation_data(df_non_class)
print("Rouge Data Non Class")
print(f'Result           : {result_non_class}')
print("---------------------------------")

---------------------------------
Sum news non clasification 4708 from 11535 XLSum


  0%|          | 0/5 [00:00<?, ?ba/s]

Rouge Data Non Class
Result           : {'rouge1': 32.6149, 'rouge2': 10.4665, 'rougeL': 25.648, 'rougeLsum': 25.6484, 'gen_len': 18.8001}
---------------------------------


In [10]:
# Goverment
df = pd.DataFrame(columns=['id', 'url', 'title', 'summary', 'text']) 
df_test_goverment= insert_new_data(goverment_politic_key, df_train_xlsum,df,"text")
print(f'lenght dataset goverment            : {len(df_test_goverment)}')

lenght dataset goverment            : 2243


In [13]:
print("---------------------------------")
result_goverment = evaluation_data(df_test_goverment)
print("Rouge Data Goverment")
print(f'Result           : {result_goverment}')
print("---------------------------------")

---------------------------------


  0%|          | 0/3 [00:00<?, ?ba/s]

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1387: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Rouge Data Goverment
Result           : {'rouge1': 29.764, 'rouge2': 9.3255, 'rougeL': 23.3987, 'rougeLsum': 23.4811, 'gen_len': 18.8047}
---------------------------------
